<a href="https://colab.research.google.com/github/h0806449f/Learn/blob/main/Tibame_dl_%E9%81%B7%E7%A7%BB%E5%AD%B8%E7%BF%92(self).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown "1DE7nQ-4QN5tlasKo8-AIlZBg3p7FyKer" --output test.zip
!unzip test.zip

In [ ]:
!gdown "1GLMtdlvdZVthqTQvi4dQfJKZet0T3Gvo" --output train.zip
!unzip train.zip

#遷移學習

In [ ]:
# VGG16 (https://keras.io/zh/applications/#vgg16)
from keras.applications.vgg16 import VGG16


vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
# include_top 使否要包含VGG16的MLP ?
# input_shape 圖片大小


vgg.summary()

In [ ]:
import glob
# 先不讀取圖片 真的要訓練時才讀取
import pandas as pd


# 創立 DataFrame 儲存路徑與資料答案
df_cat = pd.DataFrame(columns=["path", "target"])

for p in glob.glob("train/cat.*.jpg"):
    s = pd.Series([p, 0], index=["path", "target"])
    df_cat = df_cat.append(s, ignore_index=True)

df_dog = pd.DataFrame(columns=["path", "target"])

for p in glob.glob("train/dog.*.jpg"):
    s = pd.Series([p, 1], index=["path", "target"])
    df_dog = df_dog.append(s, ignore_index=True)

In [ ]:
df = pd.concat([df_cat, df_dog], axis=0, ignore_index=True)
df

In [ ]:
# 隨機讀取圖片 檢查一下
from tensorflow.keras.utils import load_img
import random


random_idx = random.randint(0, 24999)
print("下方圖片路徑:", df["path"][random_idx])
img = load_img(df["path"][random_idx])
print("下方圖片大小:", img.size)
img

In [ ]:
img = load_img(df["path"][random_idx], target_size=(224, 224))
# target_size -> 修改原本圖片大小 -> 以符合 VGG16 的 input_size

print("下方圖片大小:", img.size)
img

# 建立模型
VGG16 不是用 Sequential 堆蛋糕的方式產出  
      而是使用 model 

In [ ]:
from keras.models import Model
from keras.layers import Flatten, Dense, BatchNormalization

# CNN 
# Non-trainable params: 從 0 變為 14,714,688
# 固定那些 weight 不再訓練
for layer in vgg.layers:
    layer.trainable = False 

# BN 
BNed_layers = BatchNormalization()(vgg.output)


# MLP
temporary_vgg_output = Flatten()(BNed_layers)
# VGG16.sumary 最後輸出項有 7 * 7 * 512 = 25088 所以神經元取四位數 2的次方數

temporary_layer1 = Dense(units=2048, activation="relu")(temporary_vgg_output)

temporary_layer2 = Dense(units=256, activation="relu")(temporary_layer1)

temporary_layer3 = Dense(units=64, activation="relu")(temporary_layer2)

# 此次分類的資料集 辨識 貓還是狗 
# 如辨識結果想要 units = 1 (貓 則其他非貓)   則需要使用 sigmoid
# 如辨識結果想要 units = 2 (貓, 狗)。       則需要使用 softmax
temporary_layer4 = Dense(units=2 , activation="softmax")(temporary_layer3)

cnn = Model(inputs=vgg.input, outputs=temporary_layer4)
cnn.summary()
# 最後可以看到結果 我們自己在VGG16後面增加了多層神經元

# Compile

In [ ]:
cnn.compile(loss="categorical_crossentropy",
            optimizer="adam",
            metrics=["accuracy"])

# 切分 訓練

In [ ]:
from sklearn.model_selection import train_test_split
# 真的開始切分和訓練時 必須把pandas 轉換成 numpy array
# 捨棄pandas 前面的 index
import numpy as np
from keras.utils.np_utils import to_categorical

y_catogorical = to_categorical(df["target"])


x_train, x_test, y_train, y_test = train_test_split(np.array(df["path"]),
                                                    np.array(y_catogorical),
                                                    test_size=0.1)

# 問題的資料預處理

In [ ]:
batch_size = 100
random_idx = np.random.randint(0, x_train.shape[0], batch_size)
# 下限 0
# 上限 22500 (不包括 22500) (平常使用random.randint(0,22500) 會包含22500 但 np.random.randint 不會包含)
# batch_size 想要取幾個

In [ ]:
# Normalize相類似概念 但當時VGG16的模型訓練者 不是轉化成 0-1 or -1 - 1 
# 所以我們也需要依照他的方式 對資料預處理.  (使用誰的模型 進入該模型前的資料預處理 就要用誰的Normalize方式)
from keras.applications.vgg16 import preprocess_input
# 將上方的圖帶入模型 訓練

batch_size = 100

for i in range(20):

    random_idx = np.random.randint(0, x_train.shape[0], batch_size)
    # 下限 0
    # 上限 22500 (不包括 22500) (平常使用random.randint(0,22500) 會包含22500 但 np.random.randint 不會包含)
    # batch_size 想要取幾個

    img_list = []

    for fn in x_train[random_idx]:
        # 讀取圖片 並且使用我們偷來的模型的預處理方式 來預處理資料
        img = load_img(fn, target_size=(224, 224))
        img = preprocess_input(np.array(img))
        img_list.append(img)

    x = np.array(img_list)
    y = y_train[random_idx]

    result = cnn.train_on_batch(x, y)

    print("-----Times", i, "-----")
    print(result)

In [ ]:
batch_size = 100

random_idx = np.random.randint(0, x_test.shape[0], batch_size)

ori_img_list = []
img_list = []

for fn in x_test[random_idx]:
    img = load_img(fn, target_size=(224, 224))
    # 把原圖留下 等等可以看一下處理前 處理後
    ori_img_list.append(img)

    img = preprocess_input(np.array(img))
    img_list.append(img)

x = np.array(img_list)
y = y_test[random_idx]

cnn.evaluate(x, y)
# [16.29060935974121, 0.6650000214576721]
# loss距離 , 正確率
# 正確率不高 因為少了一個重要步驟 如下

In [ ]:
pre = cnn.predict(x).argmax(axis=-1)
pre
# argmax 比較1 or 0 的機率較大

# 發現 幾乎都預測貓 導致正確率低下
# relu 死亡問題
# 數值離 relu函式 的激活點 太遠 導致神經元跟死亡一樣
# 解決方法 : 將 cnn 的輸出值 轉化為 0-1之間 再進入 mlp
# 稱為 batch normalization

# 調整方法 - batch normalization
# 1. 輸入的數據 經過多層神經網絡 數值可能變得非常大或非常小
# 2. 非常小的數值 離relu激活點太遠 導致永遠無法激活 (relu 死亡問題)
# 3. 激活前 將數值調整到合理的範圍內

In [ ]:
# 作圖
pre != y.argmax(axis=-1)
# argmax 比較1 or 0 的機率較大

np.nonzero(pre != y.argmax(axis=-1))
# 找出那些圖示預測錯誤的